In [ ]:
!conda install -c conda-forge geopy 
!conda install -c conda-forge folium
!conda install -c conda-forge geocoder 

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


Read data from the wikipedia page and filter out data that 'Borough' was not assigned

In [2]:
rhtml = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
raw_data = rhtml[0]
raw_data = raw_data[raw_data['Borough'] != 'Not assigned']

Combine rows sharing the same 'Postcode' and merge them into one rown, separating with comma

In [3]:
raw_data = raw_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
raw_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Assign 'Beighbourhood' with 'Borough' when it's not assigned

In [4]:
ix = raw_data.loc[raw_data['Neighbourhood'] =='Not assigned'].index
raw_data['Neighbourhood'][ix] = raw_data['Borough'][ix]

Get the dimension of the dataframe

In [5]:
raw_data.shape

(103, 3)

Read in geoposition of all boroughs

In [7]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.columns = ['Postcode', 'Latitude', 'Longitude'] 
geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df = pd.merge(raw_data, geo_data, how='outer', on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
